In [1]:
import re
import emoji
import random
import os, itertools
import pandas as pd
from pandas import read_csv
from itertools import groupby

In [2]:
### Load data 
map_ = read_csv('Map_Ramen_data_with_city.csv', encoding='utf8',index_col=0)
map_ = map_.rename(columns = {'discription': 'description', 'mapReview': 'map_review', 'openTime': 'open_time', 'city_name': 'province'})
map_ = map_.sort_values(by=['store'])
map_ = map_.reset_index(drop=True)
map_ = map_.iloc[::-1]
map_ = map_.reset_index(drop=True)

# Separate main_store and detailed_store

In [3]:
### Classify MAIN or DETAILED store
map_store = list(map_['store'])
detail_store = []
stem_detail_store = []

# Filter Detailed Store, eg, XXX(新竹店), XXX-台北店, XXX-分店...
for s in map_store:
    store = s.lower()
    store = store.replace('（','(').replace('）',')').replace(u'\u3000',u'').replace('台','臺').replace('-','#')
    if ('高雄夢時代店' in store) or ('林口三井店' in store) or ('樂利店' in store) or ('臺中一中店' in store) or  \
            ('土城日月光店' in store) or ('中山店' in store) or ('高雄五福店' in store) or ('臺中逢甲店' in store) or  \
            ('師大店' in store) or ('竹北店' in store) or ('延吉街店' in store) or ('成功店' in store) or  \
            ('臺北凱撒店' in store) or ('高鐵店' in store) or ('臺中清水店' in store) or ('家樂福店' in store) or  \
            ('新光中港店' in store) or ('臺北信義店' in store) or ('臺中中友店' in store) or ('新竹勝利店' in store) or  \
            ('公館店' in store) or ('一中分店' in store) or ('大潤發店' in store) or ('小北店' in store) or  \
            ('新光三越' in store) or ('微風南山' in store):
        detail_store.append(s)
        stem_detail_store.append(s[:-3])
    elif ('express' in store):
        detail_store.append(s)
        stem_detail_store.append(s)
    else:
        if ('(' in store) and (')' in store) and (store[store.index(')')-1] == '店') \
            and ('專賣' not in store) and ('專門' not in store) and ('本店' not in store) and ('總店' not in store):
            #print(store)
            detail_store.append(s)
            stem_detail_store.append(s[:-4])
        elif ('#' in store) and ('店' in store[store.index('#'):]) and ('本店' not in store) \
            and ('專門' not in store) and ('本店' not in store) and ('總店' not in store):
            detail_store.append(s)
            stem_detail_store.append(s[:-4])
            #print(store)
        elif ('#' in store) and ('街' in store[store.index('#'):]) and '專賣' not in store \
            and ('專門' not in store) and ('本店' not in store) and ('總店' not in store):
            detail_store.append(s)
            stem_detail_store.append(s[:-4])
            #print(store)  
        elif len(store) >= 5 and (' ' == store[-4]) and ('店'== store[-1]):
            detail_store.append(s) 
            stem_detail_store.append(s[:-3])
        elif ('組' in store[-1]):
            detail_store.append(s)
            stem_detail_store.append(s[:-3])
        elif ('分店' in store) and ('臺灣分店' not in store):
            detail_store.append(s)      
            stem_detail_store.append(s[:-3])

main_store = list(set(map_store) - set(detail_store)) # Main store 285
stem_detail_store_ = stem_detail_store # Detailed store 146
stem_detail_store = list(set(stem_detail_store)) # set(detailed store) 126
all_main_store = list(set(main_store + stem_detail_store)) #  408

In [4]:
### Create 2 tables via the 'tag' column (True: detailed store; False: non-detailed store)
map_['tag'] = map_.store.isin(detail_store)
map_detail = map_.loc[map_['tag'] == True] # 146
map_main = map_.loc[map_['tag'] == False] # 285
second_main_store = list(map_main['store'])

In [5]:
### Text Preprocessing
map_store_lst = []
detail_map_store_lst = []
second_main_store_lst = []

def preprocessing(store_table, lst_append):
    for s in store_table:
        store = re.sub(r'[^\w\s]', '', s)
        store = store.replace('総','總').replace('麺','麵').replace('醤','醬').replace('ichi','ICHI') \
                        .replace('番茄','蕃茄').replace('太陽蕃茄EXPRESS','太陽蕃茄拉麵EXPRESS') \
                        .replace('#','').replace(' ','').replace(u'\u3000',u'') \
                        .replace('《','').replace('》','').replace('台湾','臺灣').replace('台','臺') \
                        .replace('らーめん','拉麵').replace('の','之') \
                        .replace('。','').replace('、',' ').replace('｜','') \
                        .replace('？','').replace('ラーメン','拉麵') \
                        .replace('(','').replace(')','').replace('·','').replace('/','') \
                        .replace('aqua2','').replace('麵鋪','麵舖') \
                        .replace('※','').replace('ぁ','あ').replace('²','2')                
        lst_append.append(store)       
    
preprocessing(all_main_store, map_store_lst) # 408
shop_sorted = sorted(map_store_lst)
preprocessing(stem_detail_store_, detail_map_store_lst) # 146
preprocessing(second_main_store, second_main_store_lst) # 285

In [6]:
### Merge 'Main store table' and 'Detailed store table' -> 'map_t2'
map_detail['stem_store'] = detail_map_store_lst
map_main['stem_store'] = second_main_store_lst
map_t2 = pd.concat([map_main, map_detail], sort=True)

/Users/yuchiacheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yuchiacheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Table 1 & Grouping Detailed Store

In [7]:
### FIRST GROUPING
util_func = lambda x: x[0]  
first_sort = [list(ele) for i, ele in groupby(shop_sorted, util_func)] 
lst = list(itertools.chain(*first_sort))

In [8]:
### SECOND GROUPING
unique = []
second_sort = []

util_func = lambda x: x[1]
for item in first_sort:
    if len(item) > 1:
        temp = sorted(item, key = util_func)
        second_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
        #second_sort.append([list(g) for _, g in itertools.groupby(item, lambda x: x[1])])
    else:
        unique.append(item[0])

second_sort_len = []

for item_1 in second_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            second_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [9]:
### THIRD GROUPING
third_sort = []

util_func = lambda x: x[2]
for item in second_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:3] == '花月嵐' \
                            or item[0][:4] == '麵屋一燈'or item[0][:3] == '麵屋濃' \
                            or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' \
                            or item[0][:4] == '鳥人拉麵' or item[0][:5] == '滝禾製麵所'\
                            or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' \
                            or item[0][:11] == '鷹流東京醬油拉麵蘭丸':
        third_sort.append([item])
    else:
        if len(item) > 1 and len(item[0]) >= 3:
            temp = sorted(item, key = util_func)
            third_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
        elif len(item) > 1 and len(item[0]) <= 2:
            third_sort.append([item])
        else:
            unique.append(item[0])
        
third_sort_len = []

for item_1 in third_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            third_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [10]:
### FOURTH GROUPING
fourth_sort = []

util_func = lambda x: x[3]
for item in third_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
                            or item[0][:3] == '花月嵐' or item[0][:4] == '麵屋一燈'\
                            or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' \
                            or item[0][:4] == '鳥人拉麵' or item[0][:11] == '鷹流東京醬油拉麵蘭丸' \
                            or item[0][:3] == '麵屋濃' or item[0][:4] == '神山拉麵' \
                            or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' \
                            or item[0][:4]== '京正拉麵'or item[0][:5] == '滝禾製麵所':
        fourth_sort.append([item])    
    else:
        if len(item) > 1 and len(item[0]) >= 4:
            temp = sorted(item, key = util_func)
            fourth_sort.append([list(ele) for i, ele in groupby(temp, util_func)])        
        elif len(item) > 1 and len(item[0]) <= 3:
            fourth_sort.append([item])
        else:
            unique.append(item[0])
        
fourth_sort_len = []

for item_1 in fourth_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            fourth_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [11]:
### FIFTH GROUPING
fifth_sort = []

util_func = lambda x: x[4]
for item in fourth_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
                            or item[0][:3] == '花月嵐' or item[0][:4] == '麵屋一燈' \
                            or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' \
                            or item[0][:4] == '鳥人拉麵' or item[0][:11] == '鷹流東京醬油拉麵蘭丸' \
                            or item[0][:3] == '麵屋濃' or item[0][:4] == '神山拉麵' \
                            or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' \
                            or item[0][:4]== '京正拉麵' or item[0][:5] == '滝禾製麵所':
        fifth_sort.append([item])    
    else:
        if len(item) > 1 and len(item[0]) > 7:
            temp = sorted(item, key = util_func)
            fifth_sort.append([list(ele) for i, ele in groupby(temp, util_func)])        
        elif len(item) > 1 and len(item[0]) <= 4:
            fifth_sort.append([item])            
        elif len(item) > 1 and len(item[0]) >= 5 and len(item[0]) <= 7:
            fifth_sort.append([item])
        else:
            unique.append(item[0])
            
fifth_sort_len = []

for item_1 in fifth_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            fifth_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [12]:
### Double Check
done = []
unclassified_group = []

for item in fifth_sort_len:
    grouped = []
    unclassified = []
    grouped.append(item[0])
    if item[0][:3] == '一風堂' or item[0][:4] == '麵屋一燈':
        done.append(item)
    elif len(item) <= 3 and len(item[0]) <= 6 and len(item[1]) <= 6:
        done.append(item)
    else:
        for i in range(len(item)-1):
            if (item[0] in item[i+1]) or (item[0][:6] in item[i+1]):
                grouped.append(item[i+1])
            else:
                unclassified.append(item[i+1])
        done.append(grouped)
        if unclassified != []:
            unclassified_group.append(unclassified)

In [13]:
### Final Check
for item in unclassified_group:
    if len(item) != 1 and item[0] != 'Mr拉麵絆臺南':
        temp_grouped = []
        temp_unclassified = []
        temp_grouped.append(item[0])
        for i in range(len(item)-1):
            if (item[0] in item[i+1]) or (item[0][:len(item[0])//2] in item[i+1]):
                temp_grouped.append(item[i+1])
            else:
                temp_unclassified.append(item[i+1])
        done.append(temp_grouped)
        #print(temp_unclassified)
        if temp_unclassified != [] and len(temp_unclassified) == 1:
            unique.append(temp_unclassified[0])
        elif temp_unclassified != [] and len(temp_unclassified) > 1:
            unique.append(temp_unclassified[:len(temp_unclassified)])
    elif len(item) != 1 and item[0] == 'Mr拉麵絆臺南':
        unique.append(item[0])
        unique.append(item[1])
    else:
        unique.append(item[0])

In [14]:
### DICTIONARY OF STORE NAME
dict_done = {}
for i in range(len(done)):
    dict_done[i] = done[i]

unique_ = []
for u in unique:
    unique_.append([u])
dict_unique = {i: unique_[j] for i,j in zip([ i for i in range(len(dict_done), len(unique)+len(dict_done))], [j for j in range(len(unique))])}

dict_main_map = dict(dict_done)
dict_main_map.update(dict_unique)

In [15]:
###  OUTPUT Table_1 'Main_Store'
table_1_name = []
for key, v in dict_main_map.items():
    table_1_name.append(v[0])

temp = {'main_store': table_1_name}
df = pd.DataFrame(temp)
df.index.name = 'store_id'
#df.to_csv('Main_Store.csv')

# Table 2

In [16]:
table_2_store_id = []
for i in list(map_t2['stem_store']):
    for key, v in dict_main_map.items():
        if i in v:
            table_2_store_id.append(key)
            break
        elif (key == list(dict_main_map.keys())[-1]):
            table_2_store_id.append('9999')
            break
            
map_t2['store_id'] = table_2_store_id
map_t2 = map_t2.drop(columns=['stem_store'])

In [17]:
### Create detail_store_id
detail_store_id = []
for i in range(1,len(table_2_store_id)+1):
    if len(str(i)) == 1:
        id_ = 'T2' + '0000' + str(i)
    elif len(str(i)) == 2:
        id_ = 'T2' + '000' + str(i)
    elif len(str(i)) == 3:
        id_ = 'T2' + '00' + str(i)
    elif len(str(i)) == 4:
        id_ = 'T2' + '0' + str(i)
    elif len(str(i)) == 6:
        id_ = 'T2' + str(i)
    detail_store_id.append(id_)
    
map_t2['detail_store_id'] = detail_store_id

In [18]:
### Create 'still_there' column
still_there = ['y' for i in range(len(table_2_store_id))]
map_t2['still_there'] = still_there

In [19]:
###  OUTPUT Table_2 'Store'
map_t2 = map_t2[['detail_store_id','store_id','store', 'still_there', 'address', 'description', 'open_time', 'latitude', 'longtitute',
       'map_review', 'region','province', 'soup', 'transport']]
#map_t2.to_csv('Store.csv', index=False)

# Add 'Others' tag

In [20]:
### Add 'Others' tag
df = read_csv('soup_data/soup_tag.csv', encoding='utf8',index_col=0)
map_t2 = read_csv('data/Store.csv', encoding='utf8',index_col=0)

soup = list(map_t2['soup'])
city = list(map_t2['province'])

tag_city = list(df.index)
others = list(df['其他'])
other_lst = []
for w in others:
    other_lst.append(w.split(','))

soup_others= []
for s, c in zip(soup, city):    
    if c in tag_city:
        for tag in other_lst[tag_city.index(c)]:
            if tag in s:
                s = s + ' #其他'
                soup_others.append(s)
                break
            elif tag == other_lst[tag_city.index(c)][-1] and (tag not in s):
                soup_others.append(s)
                break
    else:
        soup_others.append(s)

map_t2['soup'] = soup_others
map_t2 = map_t2[['store_id','store', 'still_there','address', 'description', 'open_time', 'latitude', 'longtitute',
       'map_review', 'region','province', 'soup','transport']]
#map_t2.to_csv('new_Store.csv')